In [ ]:
# Event list : ftp://ftp.swpc.noaa.gov/pub/warehouse/

In [5]:
#Libraries---
import shutil
import os
import pandas as pd
from os.path import exists
import glob
from sys import stdout
from time import sleep
import sys
import matplotlib.pyplot as plt 
from astropy.io import fits
import numpy as np
import glob
import pyCallisto as pyc
import pyCallisto_Utils as utils
import urllib.request
import tarfile
from ftplib import FTP
import ftputil
import urllib 
from urllib.request import urlopen
# import plotly.express as px

In [6]:
class variables:
    def __init__(self, **kwargs):
        self.s = "/"
        self.types = ["I","II","III","IV","V","VI"]
        self.categories = ["1","2","3"]
        for key, value in kwargs.items():
          setattr(self, key, value)

In [9]:
class Read_events(variables):

    def __init__(self,**kwargs):
        super(Read_events,self).__init__(**kwargs)
        self.path = self.dst
        self.year_int = self.year
        self.year = str(self.year)
        self.path2 = self.path+"Event list/" + self.year+"_events/*.txt"
        self.path1 = self.path
        
    def tarfile_extract(self):
        
        path = self.ftp_path+self.year+self.s
        try : #if tarfile exists       
            thetarfile = path+self.year+"_"+"events.tar.gz"
            ftpstream = urllib.request.urlopen(thetarfile)
            thetarfile = tarfile.open(fileobj=ftpstream, mode="r|gz")
            thetarfile.extractall("..\Event list")
            print("Tarfile extracted successfully")
        except: #if tarfile does not exist in ftp server, look for events folder and retrieve all .txt files
            hostname = path+self.year+"_events"
            ftpstream = urllib.request.urlopen(hostname)
            # ftpstream = urllib.request.urlretrieve(hostname,"20220101events.txt")
            with urlopen(hostname) as response:
                body = response.read().decode('utf-8')
                df = pd.DataFrame(body.split('\n')) 
            df[0] = df[0].str[55:73]
            df[0].replace('', np.nan, inplace=True)
            df.dropna(subset=[0], inplace=True)
            print(df)
        else:
            print("FTP server problem")
        
    def find(self,name, path):#return root dir and name of a given file
        for root, dirs, files in os.walk(path):
            if name in files:
                return os.path.join(root, name) 
            
    def fit_files_list(self):#this function save a list of all '.fit' files present in a given directory, in this case E drive.
        fs = []
        actual = []
        fd = []
        for root, dirs, files in os.walk(self.src): # change drive name 
            # select file name
            for file in files:
                # check the extension of files
                if file.endswith('.fit'):
                    # print whole path of files             
                    fs.append(file.split('_'))
                    fd.append(file)
                    actual.append(file.split('.'))
        d = pd.DataFrame(fs)
        ds = pd.DataFrame(actual)
        d[3],d[4] = ds[0] , fd
        d.drop(d.index[d[1].isnull()], inplace = True)
        d[5] = pd.to_datetime(d[1]).dt.date
        d[1] = pd.to_datetime(d[1] + d[2])
        d = d.set_index(d[1])
        d.drop_duplicates(subset=[4],inplace = True)
        d.index = d.index.floor('60min')
        d.drop(labels = [1],inplace = True, axis = 1)
        d.drop(d.index[d[3].str.contains(" ")], inplace = True)
        # d.to_csv("E:/CALLISTO/All_files_list.csv" , index = True)
        return d
    
    def read_files(self):# read extracted files, combine them in a dataframe
        self.tarfile_extract()
        liste = glob.glob(self.path2)
        df1 = []
        for x in range (len(liste)-1):
            #           Event   Begind    Max       End     Obs      
            colspecs = [(0, 6),(10, 16), (18, 22),(27,32),(34,37),
                        (38,40),(40,46),(48,52),(56,63),(65,73)]
            #               Q     Type   Loc     Cat/type
            df = pd.read_fwf(liste[x], colspecs=colspecs, header = None)
            df.drop(
                    labels = [0,1,2,3,4,5,6,7,8,9,10,11],
                    axis = 0,
                    inplace = True)
            df.drop(
                df.index[df[1].isnull()],
                inplace = True)
            xf = liste[x].split("\\")
            xf = xf[1].split(".")
            xf = xf[0].replace("events","")
            df[10] = xf
            df
            df1.append(df)
            continue
        return df1
    
    def search(self,index, category):#search index in parameters is for the end and begin timing column
        dfs = self.read_files()
        final = pd.concat(dfs)
        final.drop(
                final.index[final[index].isnull()],
                inplace = True)
        final.drop(
            final.index[final[index].str.contains("[a-zA-Z]")],#for removing alphabets
            inplace = True
            )
        final[11] = final[10] + final[index]
        final[11] = pd.to_datetime(final[11])
        cat_3 = final[final[8] == category]
        cat_3 = cat_3.set_index(cat_3[11])
        cat_3 =  cat_3.between_time('00:30', '15:00')
        cat_3.index = cat_3.index.floor('60min')
        return cat_3
    
    def category(self,category):#return dataframe of category passed in the parameter
        final_df = pd.concat([self.search(1,category),self.search(3,category)])
        final_df.drop(labels = [11], axis = 1, inplace = True)
        final_df.drop_duplicates(inplace = True)
        final_df
        return final_df
    
    def filter_by_time(self,g,category):# return the list of files present in the noaa solar events 
        d = g
        d.index = pd.to_datetime(d.index)
        cat = self.category(category).index
        # d.index.isin(cat)# index number
        file = d[d.index.isin(cat)]    
        return file       
    
    def categorize(self):#access types and categories, and move files in directory tree pattern
        g = self.fit_files_list()
        counter = 0
        for t in self.types:
            for c in self.categories:
                list_1 = self.filter_by_time(g,t+self.s+c)
                length = len(list_1)
                path = self.path1+self.s+self.year+self.s+t+self.s+c+self.s+"Data"
                if exists(path) == False and length:
                    os.makedirs(path)
                for x in range (len(list_1)):     
                    src = self.find(list_1[4][x], self.source)#Source Path
                    dst = path +'/'+list_1[4][x]# Destination path
                    shutil.copy(src, dst)
                    counter +=1
                    stdout.write("\r%d Files copied " % counter)
                    stdout.flush()
                    sleep(0.01)
                    continue
                    return 0   

In [11]:
# Calling Read_events_files 
kwargs = {
        "year" : 2022,
        "src" : "E:\\",
        "dst" : "E:/CALLISTO/",
        "ftp_path" : "ftp://ftp.swpc.noaa.gov/pub/warehouse/"
}
v = Read_events(**kwargs)
v.fit_files_list()

,0,2,3,4,5
1,,,,,
2016-05-04 14:00:00,GREENLAND,140001,GREENLAND_20160504_140001_63,GREENLAND_20160504_140001_63.fit,2016-05-04
2022-01-01 00:00:00,MUPK,004500,MUPK_20220101_004500_59,MUPK_20220101_004500_59.fit,2022-01-01
2022-01-01 01:00:00,MUPK,010000,MUPK_20220101_010000_59,MUPK_20220101_010000_59.fit,2022-01-01
2022-01-01 01:00:00,MUPK,011500,MUPK_20220101_011500_59,MUPK_20220101_011500_59.fit,2022-01-01
2022-01-01 01:00:00,MUPK,013000,MUPK_20220101_013000_59,MUPK_20220101_013000_59.fit,2022-01-01
...,...,...,...,...,...
2022-09-07 04:00:00,MUPK,040259,MUPK_20220907_040259_59,MUPK_20220907_040259_59.fit,2022-09-07
2022-09-07 04:00:00,MUPK,041500,MUPK_20220907_041500_59,MUPK_20220907_041500_59.fit,2022-09-07
2022-09-07 04:00:00,MUPK,043000,MUPK_20220907_043000_59,MUPK_20220907_043000_59.fit,2022-09-07


In [ ]:
class copy_files(Read_events):
    def __init__(self, **kwargs):
        super(copy_files,self).__init(**kwargs)
        self.path = self.dst
#         self.fit_files_list = self.fit_files_list
#         self.find = self.find
  
    def copy(self, li,path):
        counter = 0
        path2 = path
        if exists(path2) == False:
            os.makedirs(path2)   
        for x in range (len(li)):     
            src = self.find(li[4][x], self.src)#Source Path
            dst = path2 +'/'+li[4][x]# Destination path
            shutil.copy(src, dst)
            counter +=1
            stdout.write("\r%d Files copied " % counter)
            stdout.flush()
            sleep(0.01)
            continue   

    def yearly(self):
        path2 = self.path+str(self.year)+self.s+"All Files"
        li = self.fit_files_list()
        li = li[li.index.year == self.year]
        self.copy(li,path2)        

    def monthly(self):
        path2 = self.path+self.year+self.s+str(self.month)+ self.s+"Data"
        li = self.fit_files_list()
        li = li[li.index.year == self.year]
        li = li[li.index.month == self.month]
        self.copy(li, path2)

    def specified_date(self):
        path2 = self.path+str(self.year)+self.s+str(self.month)+ self.s+str(self.day)+self.s+"Data"
        li = self.fit_files_list()
        li = li[li.index.year == self.year]
        li = li[li.index.month == self.month]
        li = li[li.index.day == self.day]
        self.copy(li, path2)
        
    def date_range(self,start, end): 
        path = self.path+"Daily_Overview"+self.s+start +"_to_" + end+ self.s+self.s+"Data"
        per = pd.date_range(start = start, end =end, freq ='D')
        df = pd.DataFrame()
        df[1] = per.date
        df = df.set_index(df[1])
        li = self.fit_files_list()
        df = li[li[5].isin(df.index)]
        return df,path   
    
    def dir_maker(self,d,path):
        while len(d)>0:
            x = d[d[5] == d[5][1]] 
            date = x[5][1].strftime("%Y-%m-%d")#pls make it (YY-mm-dd) for future use
            path1 = path +self.s+date
            if exists(path1) == False:
                os.makedirs(path1)  
                self.copy(x,path1)
            d = d.loc[d.index.difference(x.index)]   

In [ ]:
#Copy files into a directory tree---
args = {
        "year" : 2022,
        "month" : 1,
        "day" : 1,
        "src" : "E:\\",
        "dst" : "E:/CALLISTO/"    
}
p = variables(**args)
copy_file = copy_files(p)
# d , path = copy_file.date_range('07-01-2022','09-09-2022')# mm-dd-yy
copy_file.folder_maker(d,path)

In [ ]:
class plot:
    def __init__(self,year, path):
        self.year = year
        self.path = path#"E:\CALLISTO"#data path
        self.slash = "/"

    def find(self,path):
        file_list = []
        for files in os.walk(path): 
            continue
        files = files[2]
        for file in files:
            file_size = os.path.getsize(path +"\\"+file)
            file_size = file_size/1000
            if file_size >= 200:
                file_list.append(file)
            # files = find(path)
        fit = pd.DataFrame(file_list)
#         fit[1] = fit[0].str.replace(".fit", "")
        fit[1] = fit[0].apply(lambda x: x.replace(".fit", ""))
        return fit

    def find_Data(self):      
        x = []
        start = self.path+self.slash+self.year
        for dirpath, dirnames, filenames in os.walk(start):
            for dirname in dirnames:
                if dirname == "Data":              
                    x.append(dirpath)
        return x
    
    def simple(self,src, dst,name):
            slash = "/"
            fits1 = pyc.PyCallisto.from_file(src)
            plt = fits1.spectrogram() #this will show in imshow thing
            plt.savefig(dst+slash+name+".png")
        
    def files_in_dir(self, src,dst):
        files = self.find(src)
        for file in files():
            simple(src, dst, file)
            bg(src,dst,file)
            
    def bg(self, src, dst, name):
        slash = "/"
        fits1 = pyc.PyCallisto.from_file(src)
        background_subtracted = fits1.subtract_background()
        plt = background_subtracted.spectrogram()
        plt.savefig(dst+slash+name+"_bg_sub.png")

    def simple_plot(self):
        dirs = self.find_Data()
        dirs = pd.Series(dirs)        
        for path in dirs:
            path2 = path
            path = path + "/Data"
            fit = self.find(path)
            #saving plots in this dir
            path3 = path2 + "\Plots"
            if exists(path3) == False:            
                os.mkdir(path3)    
                fit = self.find(path)
                if exists(path2) == False:
                    os.mkdir(path2)                    
                for x in range(len(fit)):    
                    fit_path = path+self.slash+fit[0][x]
                    print(x,"Processing:",fit[0][x])
                    #plot multiple files
                    dst = path3
                    name = fit[1][x]
                    self.simple(fit_path,dst,name)
                    continue
                    return False
                
    def bg_plot(self):
        
        dirs = self.find_Data()
        dirs = pd.Series(dirs)
        for path in dirs:
            path2 = path
            path = path + "/Data"
            fit = self.find(path) 
            #saving plots in this dir
            dst = path2 + "\Plots_bg_sub"   
            if exists(dst) == False:  
                os.mkdir(dst)
            for x in range(len(fit)):
              fit_path = path+self.slash+fit[0][x]
              print(x,"Processing:",fit[0][x])
              self.bg(fit_path,dst,fit[1][x])
#               print(fit_path,dst,fit[1][x])
              continue
              return False

In [ ]:
#Plot---
# pl = plot("2022", "E:\CALLISTO")# year,its parent "path"
# pl.simple_plot()
# pl.bg_plot()
# pl.bg(
#     "E:\CALLISTO\\2022\\II\\1\\Data/MUPK_20220321_052400_59.fit", 
#           "E:\CALLISTO\Specific",
#           "MUPK_20220321_052400_59"
# )
# bg = plot.bg_plot()

In [ ]:
class Daily_Overview:
    def __init__(self, **kwargs):        
         for key, value in kwargs.items():
          setattr(self, key, value)
        
    def filter(self, path, date):
        FCe = '_'+self.FC+'.fit'
        myfilter= path+self.ID+'_'+date[0:4]+date[5:7]+date[8:10]+'*'+FCe+'*'
        liste = glob.glob(myfilter)
        liste.sort()     
        return liste
    
    def files(self):
        list = []
        for root, dirs, files in os.walk(self.path):
            list.append(dirs)
        list = list[0]
        for i in range(len(list)):
            path = self.path +"\\"+ list[i]+"\\"
            date = list[i]
            liste = self.filter(path, date)
            dst = self.dst+"/"+date[0:4]+date[5:7]+date[8:10]+'.png'
            if exists(dst) == False:
                print(dst)
                try:
                    self.daily_overview(liste,date,dst) # date-code, instrument-code, focus-code
                except:
                    print ("Error, most probably one or more corrupt FIT-file(s): ",sys.exc_info())
        return False

    def daily_overview(self,liste,date,dst):  

        fig, axs = plt.subplots(4, 16, figsize=(21,8), sharex=True, sharey=True)  
        axs = axs.flatten() #from grid to flat list
        FCe = '_'+self.FC+'.fit'
#         print(liste[0], date,FCe)
#         myfilter= path+self.ID+'_'+date[0:4]+date[5:7]+date[8:10]+'*'+FCe+'*'
#         liste = glob.glob(myfilter)
#         liste.sort()
        for ffile in liste:
            if FCe in ffile:
                hdu = fits.open(ffile)
                S = hdu[0].data
                n  = 30*4 # position in time for background analysis
                dn = 7 # number of spectra to average
                spec = S[:,n:n+dn] # take a slice of a clean part
                size = spec.shape
                columns = size[0]
                background = np.mean(spec,axis=1).reshape(columns,1)
                data = S - background
                data = data.clip(-1,50) # 30 adjust color-map (0dB = blue)
                freq = hdu[1].data[0][1] # frequency axis
                date = hdu[0].header['DATE-OBS']
                Thour = int(hdu[0].header['TIME-OBS'].split(":")[0])
                Tmin  = int(hdu[0].header['TIME-OBS'].split(":")[1])
                p = int(int(Thour*60+Tmin)/15 + 0.5)
                extent = (0,1, freq[-1], freq[0])
                axs[p].imshow(data, aspect="auto", extent=extent)
                axs[p].set_xticks([])                #axs[p].axis('off')
                hdu.close()
                
        txt = '00            15             30             45              '
        xaxistxt = txt + txt + txt + txt + txt 
        fig.text(0.12, 0.07, xaxistxt, ha='left',fontsize=10.5)
        fig.text(0.055, 0.5, 'Frequency [MHz]', va='center', rotation='vertical', fontsize=15)
        fig.text(0.91, 0.84, '00-03UT', va='center', rotation='horizontal', fontsize=15)
        fig.text(0.91, 0.71, '04-07UT', va='center', rotation='horizontal', fontsize=15)
        fig.text(0.91, 0.58, '08-11UT', va='center', rotation='horizontal', fontsize=15)
        fig.text(0.91, 0.45, '12-15UT', va='center', rotation='horizontal', fontsize=15)
        fig.text(0.91, 0.32, '16-19UT', va='center', rotation='horizontal', fontsize=15)
        fig.text(0.91, 0.19, '20-23UT', va='center', rotation='horizontal', fontsize=15)
        fig.subplots_adjust(wspace=0.001, hspace=0.1)
        plt.suptitle("Full day spectra "+date+" station: "+self.ID+' with focus-code: '+self.FC, size=16)
        plt.savefig(dst)
        print(dst)

In [ ]:
# Plot daily overview plotting---
arguments = {
       "path":"E:\\CALLISTO\\Daily_Overview\\01-01-2022_to_06-30-2022\\Data",
        "FC":"59",
        "ID":"MUPK",
        "dst" : "E:\CALLISTO\Daily_Overview\\01-01-2022_to_06-30-2022\Plots_MUPK"
} 
daily = Daily_Overview(**arguments)
list1 = daily.files()

In [ ]:
class slice:
        def __init__(self,**kwargs):
               for key, value in kwargs.items():
                  setattr(self, key, value) 
            
        def find(self,name):#return root dir and name of a given file
            for root, dirs, files in os.walk(self.path):
                if name in files:
                    return os.path.join(root, name) 
    
        def slice_time(self):
                slash = "/"
#                 dirs = find_Data()    
#                 fit_path = path1+myYear+slash+types+slash+category+slash+"Data"+slash+file_name+".fit"
                file_name = os.path.splitext(self.file_name)[0]
                path1 = self.path+slash+ "Specific"+slash+'plots_for_'+file_name
                if exists(path1) == False:
                    os.makedirs(path1)
                fit_path = self.find(self.file_name)

        
                #join time axis
                joined1 = pyc.PyCallisto.from_file(fit_path)
                plt = joined1.spectrogram() #this will show in imshow thing
                plt.savefig(path1+slash+"simple_joined.png")

                # slice in frequency axis
                freq_sliced = joined1.slice_frequency_axis(self.freq1,self.freq2)
                plt = freq_sliced.spectrogram() #this will show in imshow thing
                plt.savefig(path1+slash+"freq_sliced.png")

                #do background subtraction
                background_subtracted = freq_sliced.subtract_background()
                plt = background_subtracted.spectrogram()
                plt.savefig(path1+slash+"bg_sub.png")

                #slice in time axis
                time_sliced = freq_sliced.slice_time_axis(self.begin, self.end)
                # time_sliced = freq_sliced.slice_time_axis(begin, end)
                plt = time_sliced.spectrogram() #this will show in imshow thing
                plt.savefig(path1+slash+"time_sliced.png")

                #do background subtraction
                background_subtracted = time_sliced.subtract_background()
                plt = background_subtracted.spectrogram()
                plt.savefig(path1+slash+"time_sliced_bg_sub.png")
                return 0

In [ ]:
#Plot time and frequecy slice---
params= {
    "path" : "E:\CALLISTO",
    "file_name" : "MUPK_20220417_140000_59.fit",
    "begin" : "14:07:00",
    "end" : "14:09:00",
    "freq1" : "45",
    "freq2": "200"    
}
slice = slice(**params)
slice.slice_time()

In [ ]:
class heatmap:   #Data availibility heatmap code---
    def __init__(self,year, station):
        self.year = year
        self.station = station
        
    def files(self):
        fs = []
        fd = []
        actual = []
        for root, dirs, files in os.walk(r'E:\\'):
             for file in files:
                # check the extension of files
                if file.endswith('.fit'):
                    # print whole path of files             
                    fs.append(file.split('_'))
                    fd.append(file)
                    actual.append(file.split('.'))
        d = pd.DataFrame(fs)
        ds = pd.DataFrame(actual)
        d[3],d[4] = ds[0] , fd
        d.drop(d.index[d[1].isnull()], inplace = True)
        d[5] = pd.to_datetime(d[1]).dt.date
        d[1] = pd.to_datetime(d[1] + d[2])
        d.set_index(d[1],inplace = True)
        d.drop_duplicates(subset=[4],inplace = True)
        d =  d.between_time('00:30', '15:00')
        return d 
    
    def process(self):
        ds = self.files()
        df = pd.concat([ds[5], ds[0]], ignore_index=True, axis = 1)
        station = df[df[1]==self.station]
        station = station[0]
        em = []
        liste = []
        month = ['01','02','03','04','05','06','07','08','09','10','11','12','13']
        year = int(self.year)
        for x in range(12): # filling with all the dates in a given year as a list
            if x==11:
                em.append(np.arange(self.year+'-'+month[x]+'-01',str(year+1)+'-'+month[0]+'-01',dtype='datetime64[D]'))         
            else:            
                em.append(np.arange(self.year+'-'+month[x]+'-01',self.year+'-'+month[x+1]+'-01',dtype='datetime64[D]')) 

        ds = pd.DataFrame(em)
        ds = ds.fillna(1)
        a = np.zeros((12,31))        
        for m in range(12): # filling np.zeros array with number of data files per day
            for d in range(31):
                if ds[d][m] != 1:                    
                     a[m][d] = station.loc[str(ds[d][m].date())].count()
                     continue
        return a    
    def show(self):
        data = self.process()
        label_name = "Day" # x axis label
        fig = px.imshow(data,
                        labels=dict(x=label_name, y="Months", color="No"),
                        x= list(range(0,31)),
                        text_auto=True,
                        y=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
                       )
        fig.update_xaxes(side="top")
        fig.show()
        return fig
    def save_html(self, name, dst ) :
        fig = self.show()
        fig.write_html(dst+"\\"+name+".html")  
        print(name +"_"self.year+ ".html is saved in : "+ dst)

In [ ]:
#Plot heatmap(args)
# year = "2022"
# station = "SONPK"
# dst = "E:\\CALLISTO\\Data availability"
# name = "SONPK"
# heatmap = heatmap(year,station)
# # heatmap.show()  
# heatmap.save_html(name, dst)

In [ ]:
#Advancement of temp file------> 
# import tempfile
# import shutil
# def temp_dir():
#     with tempfile.TemporaryDirectory() as tmpdir:
# #         print('Created temporary directory ', tmpdir)
# #         os.path.exists(tmpdir)
#         src = "E:\\CALLISTO\\Daily_Overview\\2021\\2021-07-03\\MUPK_20210703_004500_59.fit"
#         dst = str(tmpdir)
#         shutil.copy(src, dst)
#         for root, dirs, files in os.walk(dst):
#             print(files,tmpdir)

        
# temp_dir()

In [ ]:
#Rename dir names wrt correct date format(YY,mm,dd)
#os.chdir(r"E:\\CALLISTO\\Daily_Overview\\01-01-2022_to_06-30-2022\\Data\\")
# liste = []
# print()
# for root, dirs, files in os.walk(os.getcwd()):
#             liste.append(dirs)
# # print(liste[0])
# df = pd.DataFrame(liste[0])
# liste = liste[0]
# df[0] = pd.to_datetime(df[0])
# df[0] = df[0].astype(str)
# for x in range (len(df)):
#     os.rename(liste[x],df[0][x])
 
# print(liste[1], df[0][1])